In [1]:
# 내 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# 필요한 패키지와 모듈을 불러옴
import numpy as np
import cv2
import time
import io
import base64
from IPython.display import HTML

In [3]:
# Detection 하기 전에 원본 동영상을 Display
video = io.open('gdrive/My Drive/CV2/468 Face Landmark/video/1day2.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="50%" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
             </video>'''.format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [4]:
file_name = "gdrive/My Drive/CV2/468 Face Landmark/video/1day2.mp4"   # 원본 동영상
output_name = '1day2_output_video.mp4'    # detection 된 output 동영상

In [5]:
!pip install mediapipe

     |████████████████████████████████| 36.1MB 114kB/s 


In [6]:
import mediapipe as mp
mp_face_mesh = mp.solutions.face_mesh

In [7]:
# Prepare DrawingSpec for drawing the face landmarks later.
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

In [8]:
def detectAndDisplay(frame):
    with mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=5,
        min_detection_confidence=0.5) as face_mesh:
      # Convert the BGR image to RGB and process it with MediaPipe Face Mesh.
      results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

      # Draw face landmarks of each face.
      if results.multi_face_landmarks:
          for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACE_CONNECTIONS,
                landmark_drawing_spec=drawing_spec,
                connection_drawing_spec=drawing_spec)

          # video 를 disk 에 output 하기 위해 writer 를 초기화한다.
          global writer
          if writer is None and output_name is not None:
              fourcc = cv2.VideoWriter_fourcc(*"MJPG")
              writer = cv2.VideoWriter(output_name, fourcc, 30,
                      (frame.shape[1], frame.shape[0]), True)
              
          # disk 에 frame 을 write 합니다.
          if writer is not None:
              writer.write(frame)

In [9]:
# 원본 동영상에서 video stream을 읽어온다.
cap = cv2.VideoCapture(file_name)
writer = None
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        break
    detectAndDisplay(frame)

--(!) No captured frame -- Break!
